In [3]:
import pandas as pd
import numpy as np
import os
from scipy.integrate import cumulative_trapezoid

# ==========================================
# 1. 사용자 설정 (Path 및 파라미터)
# ==========================================
# 폴더 경로는 사용자의 환경에 맞게 유지됩니다.
folder_path = r"C:\Users\user\Desktop\Gibeom\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\8. Gibeom touch\260219_Cap별 peak 하나 계형\csv파일"

output_filename = "00Results_Summary_Voltage.csv"

fraction_percent = 2  # baseline 추정 비율 (%)
R_load = 40e6         # 부하 저항: 40 M옴 (40 * 10^6)



In [4]:
# ==========================================
# 2. 전하 적분 함수 정의 (기존 로직 유지)
# ==========================================
def segment_and_integrate(current_ua, time, df_full, threshold=0.01, scale_factor=1e3):
    """
    current_ua: uA 단위의 전류 배열
    scale_factor: 1e3을 곱해 uC -> nC로 변환
    """
    if len(time) < 1:
        return 0.0
    total_charge = 0.0
    group_start = 0
    time_diffs = np.diff(time)
    
    for i, dt in enumerate(time_diffs):
        if dt > threshold:
            segment_t = time[group_start:i+1]
            segment_i = current_ua[group_start:i+1]
            if len(segment_t) > 1:
                total_charge += scale_factor * cumulative_trapezoid(segment_i, segment_t, initial=0)[-1]
            elif len(segment_t) == 1:
                dt_est = 1e-6
                t0 = segment_t[0]
                if t0 in df_full["Time"].values:
                    idx = df_full.index[df_full["Time"] == t0].tolist()[0]
                    if 0 < idx < len(df_full) - 1:
                        dt_est = (df_full.iloc[idx + 1]["Time"] - df_full.iloc[idx - 1]["Time"]) / 2
                total_charge += scale_factor * segment_i[0] * dt_est
            group_start = i + 1
            
    if group_start < len(time):
        segment_t = time[group_start:]
        segment_i = current_ua[group_start:]
        if len(segment_t) > 1:
            total_charge += scale_factor * cumulative_trapezoid(segment_i, segment_t, initial=0)[-1]
        elif len(segment_t) == 1:
            dt_est = 1e-6
            t0 = segment_t[0]
            if t0 in df_full["Time"].values:
                idx = df_full.index[df_full["Time"] == t0].tolist()[0]
                if 0 < idx < len(df_full) - 1:
                    dt_est = (df_full.iloc[idx + 1]["Time"] - df_full.iloc[idx - 1]["Time"]) / 2
            total_charge += scale_factor * segment_i[0] * dt_est
            
    return total_charge

# ==========================================
# 3. 폴더 순회 및 데이터 처리
# ==========================================
results_list = []
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv') and f != output_filename])

print(f"총 {len(file_list)}개의 파일을 처리합니다. (저항: {R_load/1e6} MOhms)")

for filename in file_list:
    full_path = os.path.join(folder_path, filename)
    
    try:
        # 데이터 로드 (Time, Voltage)
        df = pd.read_csv(full_path, header=None, skiprows=30)
        df.columns = ['Time', 'Voltage']
        df["Time"] = pd.to_numeric(df["Time"], errors='coerce')
        df["Voltage"] = pd.to_numeric(df["Voltage"], errors='coerce')
        df = df.dropna(subset=['Time', 'Voltage'])

        # 1. Baseline Correction (전압 기준)
        v_raw = df['Voltage'].values
        n_total = len(v_raw)
        n_fraction = int(n_total * (fraction_percent / 100))
        
        v_offset = np.mean(np.concatenate([v_raw[:n_fraction], v_raw[-n_fraction:]]))
        v_offset = 0.0 if np.isnan(v_offset) else v_offset
        
        # 2. 보정된 전압 및 전류 변환
        # Cor_current_uA = (V_corrected / R) * 10^6
        df['Cor_voltage'] = df['Voltage'] - v_offset
        df['Cor_current_uA'] = (df['Cor_voltage'] / R_load) * 1e6
        
        # 3. 최대/최소 전압 추출
        max_v = df['Cor_voltage'].max()
        min_v = df['Cor_voltage'].min()
        
        # 4. 전하 적분 (양수/음수 분리)
        p_df = df[df["Cor_current_uA"] > 0]
        n_df = df[df["Cor_current_uA"] < 0]
        
        p_charge = segment_and_integrate(p_df["Cor_current_uA"].values, p_df["Time"].values, df)
        n_charge = segment_and_integrate(n_df["Cor_current_uA"].values, n_df["Time"].values, df)
        
        results_list.append({
            "File Name": filename,
            "Max Voltage (V)": max_v,
            "Min Voltage (V)": min_v,
            "Max Charge (nC)": p_charge,
            "Min Charge (nC)": n_charge,
            "V_Offset (V)": v_offset
        })
        print(f"완료: {filename}")
        
    except Exception as e:
        print(f"오류 ({filename}): {e}")

# ==========================================
# 4. 결과 저장
# ==========================================
summary_df = pd.DataFrame(results_list)
summary_df = summary_df.round(4) # 전하량이 작을 수 있어 소수점 4자리까지 확장

output_path = os.path.join(folder_path, output_filename)
summary_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("-" * 30)
print(f"모든 작업이 완료되었습니다. 결과 파일: {output_filename}")
print(summary_df.head())

총 15개의 파일을 처리합니다. (저항: 40.0 MOhms)
완료: 01._0nF_TENG.csv
완료: 02._1nF_TENG.csv
완료: 03._3.3nF_TENG.csv
완료: 04._4.7nF_TENG.csv
완료: 05._6.8nF_TENG.csv
완료: 06._8.2nF_TENG.csv
완료: 07._10nF_TENG.csv
완료: 08._22nF_TENG.csv
완료: 09._33nF_TENG.csv
완료: 10._47nF_TENG.csv
완료: 11._68nF_TENG.csv
완료: 12._82nF_TENG.csv
완료: 13._220nF_TENG.csv
완료: 14._1_Cap.csv
완료: 15._2_Cap.csv
------------------------------
모든 작업이 완료되었습니다. 결과 파일: 00Results_Summary_Voltage.csv
            File Name  Max Voltage (V)  Min Voltage (V)  Max Charge (nC)  \
0    01._0nF_TENG.csv          58.7958          -0.1042          24.7797   
1    02._1nF_TENG.csv          17.3530          -0.0111          25.3130   
2  03._3.3nF_TENG.csv           6.8190          -0.0088          25.4507   
3  04._4.7nF_TENG.csv           5.0723          -0.0065          25.5342   
4  05._6.8nF_TENG.csv           3.7910          -0.0087          24.7730   

   Min Charge (nC)  V_Offset (V)  
0          -0.1093        0.0792  
1          -0.0439        0.0